In [4]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import pandas as pd
import requests
import ast
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [5]:
from fetch_trends import hashtag_trend_info

In [13]:
url = f'https://ads.tiktok.com/business/creativecenter/trends/home/pc/en'
page = requests.get(url)
soup = BeautifulSoup(page.text, features='lxml')

trending_hashtag = []
trending_music = []
trending_creator = []

# get hashtag info
for s in soup.find_all('div', id='hashtagItemContainer'):
    tag = s.find('span', {"class": "titleText--qKHbP"}).text
    stats = s.find_all('span', {"class": "item-value--VAdnq"})
    posts = stats[0].text
    views = stats[1].text
    trending_hashtag.append([tag, posts, views])

# get music info
for s in soup.find_all('div', {"class": "music-name-wrap--FPaLL music-name-wrap--X6m9u"}):
    trending_music.append(s.text)

# get creator info
for s in soup.find_all('div', id='creatorItemContainer'):
    tag = s.find('span', {"class": "music-name--LR+1s music-name--bEzZ1"}).text
    stats = s.find_all('span', {"class": "creator-data-value--CwFQt creator-data-value--S-K2V"})
    posts = stats[0].text
    views = stats[1].text
    trending_creator.append([tag, posts, views])

hashtag = pd.DataFrame(trending_hashtag, columns=['tag', 'posts_count', 'views_count'])
music = pd.DataFrame(trending_music, columns=['music'])
creator = pd.DataFrame(trending_creator, columns=['creator', 'followers_count', 'likes_count'])

find_content = soup.find('script', id="__NEXT_DATA__")
str_content = str(find_content).split('<')[1].split('>')[1]

json_content = json.loads(str_content)['props']['pageProps']

if 'data' in json_content.keys():
    json_content = json_content['data']

hashtag_content = json_content['hashtags']
music_content = json_content['trendMusic']
creator_content = json_content['creatorList']
tiktok_content = json_content['tiktoks']

# gather all data
hashtag = hashtag.merge(pd.DataFrame.from_records(hashtag_content), left_index=True, right_index=True)
music = music.merge(pd.DataFrame.from_records(music_content), left_index=True, right_index=True)
creator = creator.merge(pd.DataFrame.from_records(creator_content), left_index=True, right_index=True)
tiktok = pd.DataFrame.from_records(tiktok_content)

In [31]:
json_content['hashtags'] == []

True

In [12]:
hashtag['creators_examples'] = hashtag.apply(lambda x: ([e['nickName'] for e in x['creators']]), axis=1)


ValueError: Wrong number of items passed 3, placement implies 1

In [ ]:
hashtag